In [ ]:
!pip install flair

     |████████████████████████████████| 286kB 4.1MB/s 
     |████████████████████████████████| 19.7MB 1.4MB/s 
     |████████████████████████████████| 798kB 49.5MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 1.2MB 52.1MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 2.5MB 56.6MB/s 
     |████████████████████████████████| 983kB 61.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 41.3MB/s 
     |████████████████████████████████| 901kB 42.0MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9705 sha256=37564e5d932e22ae34d138e20266d45aa9bd628e1065be05c70c9422218b8dd5
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for mpld3: filename

In [ ]:
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings
import numpy as np

In [ ]:
embedding_model = TransformerDocumentEmbeddings('bert-large-uncased', fine_tune = False)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import pandas as pd
def read_comments_from_dataset():
  return pd.read_csv("CommentsApril2017_DEV_LARGE.csv")[["commentID", "commentBody", "commentSequence", "articleID", "parentID"]]

In [ ]:
comments = read_comments_from_dataset()
len(comments)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (15,16,17,21,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


108487

In [ ]:
from tqdm import tqdm
embeddings = []

count = 0

for comment in tqdm(comments.itertuples()):
  try:
    comment_sentence = Sentence(comment.commentBody)
    embedding_model.embed(comment_sentence)
  except:
    comment_sentence = Sentence("None")
    embedding_model.embed(comment_sentence)
  embeddings.append({"ID": comment.commentID,
                    "article_id": comment.articleID, 
                     "commentBody": comment.commentBody,
                     "embedding": comment_sentence.embedding})
  count += 1
  if count > 5000:
    del embedding_model
    del comment_sentence
    embedding_model = TransformerDocumentEmbeddings('bert-large-uncased', fine_tune = False)
    count = 0

4997it [02:01, 45.87it/s]Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
10000it [04:15, 33.40it/s]Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: [

In [ ]:
import torch
torch.save(embeddings, "embedded_comments_bert_large_uncased.pt")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
torch.save(embeddings, "gdrive/MyDrive/Research/embedded_comments_bert_large_uncased.pt")